## 実行位置の調整 

In [2]:
%cd ..

E:\システムトレード入門\predict_git_workspace


## インポート 

In [16]:
import csv
from pathlib import Path
from tqdm.notebook import tqdm
import datetime
import shutil
import time
import numpy as np

In [4]:
from get_stock_price import YahooFinanceStockLoaderMin

In [5]:
from get_stock_price import StockDatabase

In [6]:
import pandas as pd

## 自作tqdm 

tqdmを使っていてなぜか止まることがあったので，こちらで行うことにする．

In [107]:
def mytqdm(an_iter):
    """
    tqdmを模したジェネレータ．イテレーション可能なオブジェクトを引数とする．
    an_iter: any of iterable
        進捗度を出力するイテレータ
    """
    length = len(an_iter)
    my_iter = iter(an_iter)
    counter = 0
    start_time = time.time()
    old_start_time = time.time()
    while True:
        try:
            new_start_time = time.time()
            next_iter = next(my_iter)  # 終了時はここでエラーが出る
            counter += 1
            one_take_time = new_start_time - old_start_time
            print("\r{}/{}, [{:.3f} sec]".format(counter,length ,one_take_time), end="")
            old_start_time = new_start_time
            yield next_iter
            
        except StopIteration as e:  # StopIterationErrorのみ通す
            counter += 1
            end_time = time.time()
            all_take_time = end_time - start_time
            print("\r{}/{}, mean [{:.3f} sec]".format(length, length, all_take_time/counter))
            return None  # StopIterationErrorを起こす

In [108]:
for i in mytqdm(range(100)):
    time.sleep(1)

100/100, mean [0.991 sec]


## 個別株のインサートを行うクラス

メモリの点から，一つ一つの銘柄ごとにデータベースにインサートしている．

In [109]:
class CsvKobetsuInsert():
    """
    csvファイルで読み込んだ銘柄のリストをもとに，StockDataBaseにデータをインサートしていく．メモリの点から，一つ一つの銘柄ごとにインサートする．
    """
    def __init__(self, csv_path, stock_loader, stock_db, stock_group="nikkei_255", use_tempfile=False):
        """
        csv_path: pathlib.Path
            csvファイルのパス(内容は銘柄コード，銘柄名)
        stock_loader: YahooFinanceStockLoaderMin
            データローダ．今のところYahooFinanceを利用したもののみ
        stock_db: StockDataBase
            データベース
        stock_group: str
            銘柄グループの名前．csvファイルに対応させる
        use_tempfile: bool
            tempfileを利用するかどうか．tempfileを利用すると，プログラムが途中で終了した場合そこからスタートできる．
        """
        self.csv_path = Path(csv_path)
        self.stock_loader = stock_loader
        self.stock_db = stock_db
        self.stock_codes = pd.read_csv(self.csv_path, header=0)  # 自分で作成
        
        self.stock_group = stock_group
        if len(self.stock_codes) < 1:
            print("csv cannot read")
        self.use_tempfile = use_tempfile
        self.complete_stock_name = None

    def tempfile_check(self):
        # 以下，処理が停止してしまった際に，作業の完了位置を読み込む
        csv_path_file_name = self.csv_path.stem
        tempfile_path = self.csv_path.parent / Path("{}_tempfile.tmp".format(csv_path_file_name))
        if tempfile_path.exists():
            with open(tempfile_path, "r") as f:
                reader = csv.reader(f)

                #dateについて取得, 現在時間との差が一日以内かどうか判定
                datetime_list = next(reader)  # [datetime,実際の日時の文字列]
                tempfile_date = datetime.datetime.strptime(datetime_list[1], "%Y-%m-%d %H:%M:%S")
                if datetime.datetime.now() - tempfile_date >= datetime.timedelta(days=1):
                    print("tempfile is not recent date, please check tempfile")

                complete_stock_name_list = next(reader)  # [complete_stock_name, 実際に終了した銘柄コード]
                self.complete_stock_name = complete_stock_name_list[1]
            
            # tempfileの削除
            tempfile_path.unlink()
            
        else:
            self.complete_stock_name = None
            print("cannot read tempfile")
            

    def __call__(self):
        print("[{}] {}_kobetsu_insert start".format(str(datetime.datetime.now()), self.stock_group))

        stock_codes_array = self.stock_codes.loc[:,"code"].values.astype("str")
        if self.use_tempfile:
            # テンプファイルの読み込み
            self.tempfile_check()
            if self.complete_stock_name is not None:
                # self.complete_stock_nameよりインデックスの大きい部分array
                complete_index = np.where(stock_codes_array==self.complete_stock_name)[0].item()
                if complete_index != len(stock_codes_array)-1:  # complete_indexが最後のインデックスでない場合
                    stock_codes_array = stock_codes_array[complete_index+1:]  # スライシングはintegerでないといけないため，item
                    pre_stock_code = None  # 完了した最後のstock_code
                else:
                    stock_codes_array = []  # 空リスト
                    pre_stock_code = self.complete_stock_name  # 最後の銘柄コード
                # 途中から始まることを明示
                print("[{}] already {} item inserted, start from {}".format(str(datetime.datetime.now()), complete_index+1, self.complete_stock_name))  # indexは0スタートであるため，+1
        
        
        #for stock_code in tqdm(self.stock_codes.loc[:,"code"].values):
        for stock_code in mytqdm(stock_codes_array):
            stock_name = str(stock_code) + ".T"  # これはyahooが前提
            self.stock_loader.set_stock_names(stock_name)
            # エラー処理
            try:
                df = stock_loader.load()
                if df is not None:
                    self.stock_db.upsert(df, item_replace_type="replace_null")
                else:
                    print("\n[{}] cannot get {}".format(str(datetime.datetime.now()), stock_code))
                
            except BaseException as e:  # おそらく強制終了か通信のエラー
                # tempfileの作成
                csv_path_file_name = self.csv_path.stem
                self.tempfile_path = self.csv_path.parent / Path("{}_tempfile.tmp".format(csv_path_file_name))
                with open(self.tempfile_path, "w", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(["datetime", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")])  # 現在時刻を記述
                    writer.writerow(["complete_stock_name", pre_stock_code])  # 完了した銘柄コードを記述
                    
                raise e  # エラーが分かるようにraiseしておく
            # pre_stock_nameを更新
            pre_stock_code = stock_code

        print("[{}] {}_kobetsu_insert end".format(str(datetime.datetime.now()), self.stock_group))
        
        # 終了時にテンプファイルを作成
        csv_path_file_name = self.csv_path.stem
        self.tempfile_path = self.csv_path.parent / Path("{}_tempfile.tmp".format(csv_path_file_name))
        with open(self.tempfile_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["datetime", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")])  # 現在時刻を記述
            writer.writerow(["complete_stock_name", pre_stock_code])  # 完了した銘柄コードを記述
    
    def close(self):
        # 全ての__call__が終了したとき,これでtempfileが残らなくなる
        self.tempfile_path.unlink()

In [110]:
class FunctionComposer():
    """
    callableなオブジェクトをまとめてcallする．closeメソッドも考慮できる．
    """
    def __init__(self, function_list, use_close=False):
        """
        function_list: list of function
            関数のリスト
        use_close: bool
            closeメソッドを適用するかどうか
        """
        self.function_list = function_list
        self.use_close = use_close
        
    def __call__(self):
        """
        全ての関数の実行
        """
        for func in self.function_list:
            func()  # 関数の実行
        if self.use_close:
            self.close()
            
    def close(self):
        """
        closeを実行する．hasattrで確認すべき？
        """
        [func.close() in self.function_list]

In [111]:
#db_path = Path("db/big_sample_db/") / Path("stock.db")
db_path = Path("db/stock_db") / Path("stock.db")

stock_db = StockDatabase(db_path, column_upper_limit=1000, table_name_base="table", database_frequency="T")
nikkei_code_file_path = Path("get_stock_price/nikkei225.csv")
tosho_code_file_path = Path("get_stock_price/tosho.csv")

stock_loader = YahooFinanceStockLoaderMin(None, past_day=5, stop_time_span=2.0, is_use_stop=True)  #一つ一つストップしながら

nikkei_kobetsu_insert = CsvKobetsuInsert(nikkei_code_file_path, stock_loader, stock_db, stock_group="nikkei_255", use_tempfile=True)
tosho_kobetsu_insert = CsvKobetsuInsert(tosho_code_file_path, stock_loader, stock_db, stock_group="tosho_1", use_tempfile=True)

# func_composer = FunctionComposer([nikkei_kobetsu_insert, tosho_kobetsu_insert])

## 日経255銘柄の株価データをデータベースにインサート 

In [114]:
nikkei_kobetsu_insert()

[2020-12-01 01:13:48.539837] nikkei_255_kobetsu_insert start
[2020-12-01 01:13:48.606652] already 219 item inserted, start from 2413
0/0, mean [0.000 sec]
[2020-12-01 01:13:48.609645] nikkei_255_kobetsu_insert end


## 東証一部上場銘柄の取得

In [11]:
stock_loader = YahooFinanceStockLoaderMin(None, past_day=5, stop_time_span=2.0, is_use_stop=False)  #一つ一つストップする
tosho_kobetsu_insert = CsvKobetsuInsert(tosho_code_file_path, stock_loader, stock_db, stock_group="tosho_1")
tosho_kobetsu_insert()

[2020-11-05 17:45:38.176312] tosho_1_kobetsu_insert start
641/4044, [2.101 sec]Not Found: No data found, symbol may be delisted
1026/4044, [4.807 sec]Not Found: No data found, symbol may be delisted
4044/4044, mean [5.114 sec]
[2020-11-05 23:30:24.063812] tosho_1_kobetsu_insert end


## データベースのバックアップクラス 

In [92]:
class PyBackUp():
    """
    指定したファイル・フォルダをバックアップする．保存形式は元と同じあるいはzip形式．
    zip形式にはLZMA形式を利用する
    """
    def __init__(self, source_path, backup_path, back_number=6, is_use_text=True, to_zip=False):
        """
        source_path: str or pathlib.Path
            バックアップしたいソースのパス．ファイルでもディレクトリでも良い．
        backup_path: str or pathlib.Path
            バックアップ先のディレクトリのパス．そのディレクトリにsource_pathに対応したフォルダを作成する．
        back_number: int
            バックアップファイルの個数．
        is_use_text: bool
            バックアップファイルの管理にcsvを使うかどうか
        to_zip: bool
            zip形式で保存するかどうか
        """
        source_path = Path(source_path)
        if not source_path.exists():
            raise ValueError("This path does not exists")
        self.source_path = source_path
        self.source_name = source_path.name  # ファイル名
        self.source_stem = source_path.stem  # 拡張子を除いたファイル名

        backup_path = Path(backup_path)
        if not backup_path.exists():
            backup_path.mkdir()

        self.backup_path = backup_path

        self.back_number = back_number
        self.backup_counter = -1  # 0からスタートするように
        
        self.to_zip = to_zip
        if is_use_text:
            self.read_backup_data()

    def back_up(self):
        print("[{}] backup start.".format(str(datetime.datetime.now())))
        self.backup_counter += 1
        
        backup_number = int((self.backup_counter)%self.back_number)  # 保存するディレクトリに対応
        backup_dir_name = "back_up_" + str(backup_number)

        backup_dir_path = self.backup_path / Path(backup_dir_name)
        backup_dst_path = backup_dir_path / Path(self.source_name)  # 実際に保存するパス

        if not backup_dir_path.exists():  # バックアップファイルのディレクトリが存在しない場合
            backup_dir_path.mkdir(parents=True)  # ディレクトリを作成
            
        if backup_dst_path.exists():  # すでにバックアップファイルが存在する場合
            if backup_dst_path.is_file():  # ファイルの場合
                backup_dst_path.unlink()  # 削除
            elif backup_dst_path.is_dir():  # ディレクトリの場合
                shutil.rmtree(backup_dst_path)

        # バックアップファイルのコピー
        if self.to_zip:  # zip
            backup_dst_path = backup_dst_path.with_suffix(".zip")  # zipとつける
            make_zip(source_path=self.source_path, zip_path=backup_dst_path)
            
        else: #zipでなくコピー
            if self.source_path.is_file():
                shutil.copyfile(src=self.source_path, dst=backup_dst_path)
            elif self.source_path.is_dir():
                shutil.copytree(src=self.source_path, dst=backup_dst_path)
        
        backup_data_text_path = backup_dir_path / Path("data.csv")
        if not backup_data_text_path.exists():  # バックアップデータの詳細を書いたテキストファイル
            backup_data_text_path.touch(exist_ok=True)

        # バックアップデータの書き込み・書き換え
        with open(backup_data_text_path, "w", newline="") as f:
            writer = csv.writer(f)
            backup_time = datetime.datetime.now()
            writer.writerow(["date", backup_time.strftime("%Y-%m-%d %H:%M:%S")])
            writer.writerow(["dir_number", backup_number])
            writer.writerow(["backup_count", self.backup_counter])


        print("[{}] back up db_file {}".format(str(backup_time),str(backup_number)))
        print("[{}] backup end.".format(str(datetime.datetime.now())))
        return backup_dst_path

    def read_backup_data(self):
        backup_datetime_list = []
        backup_counter_list = []

        for backup_dir in self.backup_path.iterdir():
            # バックアップファイルの存在確認
            if self.to_zip: #Zipの場合
                backup_file_path = backup_dir / Path(self.source_stem).with_suffix(".zip")  # xz.tarを前提
            else:
                backup_file_path = backup_dir / Path(self.source_name)  
                
            if backup_file_path.exists():  # バックアップファイルが存在する場合
                backup_data_text_path = backup_dir / Path("data.csv")
                if backup_data_text_path.exists():
                    # バックアップデータの読み込み
                    with open(backup_data_text_path, "r") as f:
                        reader = csv.reader(f)
                        
                        datetime_list = next(reader)
                        backup_datetime = datetime.datetime.strptime(datetime_list[1],"%Y-%m-%d %H:%M:%S")
                        
                        next(reader)  # この行はいらなかったかも

                        counter_list = next(reader)
                        backup_counter_list.append(counter_list[1])

                        backup_datetime_list.append(backup_datetime)

        # バックアップデータが存在する場合
        if len(backup_datetime_list) > 0:
            # 最近のインデックスを求める, maxカウンターでもいいけど念のため
            def get_timestamp(datetime):
                return datetime.timestamp()
            max_date = max(backup_datetime_list, key=get_timestamp)
            max_date_index = backup_datetime_list.index(max_date)
            
            self.backup_counter = int(backup_counter_list[max_date_index])

### データベースのバックアップ 

In [95]:
db_path = Path("db/big_sample_db") / Path("stock.db")
backup_path = Path("db/backup")
db_backup = PyBackUp(source_path=db_path, backup_path=backup_path, back_number=5)

In [96]:
db_backup.back_up()

[2020-11-30 22:49:23.125374] backup start.
[2020-11-30 22:49:29.972103] back up db_file 3
[2020-11-30 22:49:30.002994] backup end.


WindowsPath('db/backup/back_up_3/stock.db')